# Интро и полезная литература

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [1]:
cd ..

/Users/polina/WebstormProjects/ml-bug-detector


Заимпортируем всё, что нам пригодится в дальнейшем

In [2]:
from shared.helpers.word2VecPreprocessing import filter_tokens_fo_Word2Vec

Также внешние библиотеки подключим сразу

In [3]:
from os import walk
import pandas as pd
import json
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

# Строим модель

In [25]:
# папка, в которой лежат нужные нам json
FOLDER_WITH_JSONS = 'data'

# как начинается имя json, в которых лежат токенизированные скрипты
TOKENIZED_JSON_START_WITH_PREFIX = 'tokenized-scripts'

Так как токенизация была разбита на микрозадачи (чтобы хватило мощностей компьютера), то нужно найти все json и объединить их в одну

In [32]:
# filenames - Список всех файлов в интересуемой нами директории
fdirpath, dirnames, filenames = list(walk(FOLDER_WITH_JSONS))[0]

tokenized_scripts_json = list(
    filter(
        lambda script_name: script_name.startswith(TOKENIZED_JSON_START_WITH_PREFIX),
        filenames
    )
)

In [4]:
tokenizedScriptsJson = open(pathToTokenizeScriptsJson) 
tokenized_scripts_dictionary = json.load(tokenizedScriptsJson)

FileNotFoundError: [Errno 2] No such file or directory: 'data/tokenized-scripts.json'

In [ ]:
for fileName in tokenized_scripts_dictionary.keys():
    tokenized_scripts_dictionary[fileName] = filter_tokens_fo_Word2Vec(tokenized_scripts_dictionary[fileName])

In [ ]:
tokenized_scripts_list_of_lists = list(tokenized_scripts_dictionary.values())

Теперь бы нужно настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [ ]:
word2VecModel = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=tokenized_scripts_list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=5,
    
    # финальный размер словаря
    max_final_vocab=10000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 5, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

# Сохранение и загрузка модели

In [ ]:
word2VecModel.save('word2Vec/word2VecModel')

In [ ]:
from gensim.models.word2vec import Word2Vec
word2VecModel = Word2Vec.load('word2Vec/word2VecModel')

In [ ]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

Таки да.

- clearTimeout - обрывает запланированный вызов функции
- timeout - через какое время исполнить функцию (один из агрументов всегда)
- cb - это callback-функции (один из агрументов всегда)

In [ ]:
print('В нашем словаре теперь', len(word2VecModel.wv.vocab), 'слов')